# Results p. 3
## Fixation Discriminability
### How sensitive are detectors to fixation onsets/offsets?

In [1]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.hfc._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


In [2]:
FIG_ID, IS_SUPP = 5, False
EVENT_NAME, EVENT_LABEL = "Fixation", 1       # EventLabelEnum.FIXATION.value
THRESHOLD = 5   # samples
METRIC = peyes.constants.D_PRIME_STR        # can also be `peyes.constants.F1_STR` or `peyes.constants.CRITERION_STR`

GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH
LINE_WIDTH, ERROR_WIDTH = 2 * ZERO_LINE_WIDTH, ZERO_LINE_WIDTH
MARKER_SIZE = LINE_WIDTH * 2

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=22, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=18, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=16, color=FONT_COLOR)
AXIS_TITLE_STANDOFF = 2

## Load Data

In [3]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

# drop human annotators that aren't GT1 or GT2 (and aren't detectors)
annotators_to_drop = [ann for ann in csdt_metrics.columns.get_level_values(u.PRED_STR).unique() if ann not in [GT1, GT2] and ann not in DETECTORS.keys()]
csdt_metrics.drop(columns=annotators_to_drop, level=u.GT_STR, inplace=True)
csdt_metrics.drop(columns=annotators_to_drop, level=u.PRED_STR, inplace=True)

csdt_metrics

trial_id                                       1                       \
gt                                             MN                       
pred                                          ivt        RA        nh   
channel_type metric           threshold                                 
onset        recall           0          0.027778  0.416667  0.083333   
             precision        0          0.009174  0.375000  0.088235   
             f1               0          0.013793  0.394737  0.085714   
             false_alarm_rate 0          0.024183  0.005598  0.006941   
             d_prime          0          0.059638  2.326102  1.077291   
...                                           ...       ...       ...   
offset       precision        20         0.220183  0.875000  0.852941   
             f1               20         0.331034  0.921053  0.828571   
             false_alarm_rate 20              NaN  0.067746  0.067746   
             d_prime          20              NaN  3.407295  2.354423   
             criterion        20              NaN -0.210858  0.315577   

trial_id                                                               \
gt                                                                      
pred                                     remodnav      ivvt       idt   
channel_type metric           threshold                                 
onset        recall           0          0.055556  0.000000  0.027778   
             precision        0          0.048780  0.000000  0.030303   
             f1               0          0.051948       NaN  0.028986   
             false_alarm_rate 0          0.008733  0.013883  0.007165   
             d_prime          0          0.783544 -1.317884  0.534365   
...                                           ...       ...       ...   
offset       precision        20         0.634146  0.177419  0.636364   
             f1               20         0.675325  0.224490  0.608696   
             false_alarm_rate 20         0.203239  0.691011  0.162591   
             d_prime          20         1.419565 -1.007207  1.194294   
             criterion        20         0.120327  0.004885  0.386718   

trial_id                                                               \
gt                                                                 RA   
pred                                      engbert      idvt       ivt   
channel_type metric           threshold                                 
onset        recall           0          0.055556  0.027778  0.025000   
             precision        0          0.031250  0.031250  0.009174   
             f1               0          0.040000  0.029412  0.013423   
             false_alarm_rate 0          0.013883  0.006941  0.024204   
             d_prime          0          0.607367  0.545779  0.013798   
...                                           ...       ...       ...   
offset       precision        20         0.562500  0.781250  0.229358   
             f1               20         0.720000  0.735294  0.335570   
             false_alarm_rate 20         0.379379  0.094845       NaN   
             d_prime          20         2.669370  1.819987       NaN   
             criterion        20        -1.033237  0.401505       NaN   

trial_id                                           ...        10            \
gt                                                 ...        MN             
pred                                           nh  ...   engbert      idvt   
channel_type metric           threshold            ...                       
onset        recall           0          0.125000  ...  0.121951  0.024390   
             precision        0          0.147059  ...  0.102041  0.035714   
             f1               0          0.135135  ...  0.111111  0.028986   
             false_alarm_rate 0          0.006499  ...  0.009859  0.006050   
             d_prime          0          1.333457  ...  1.166389  0.538724   
...       

## Multi-Threshold Figure
### (not in the paper)
This figure shows the `METRIC` values at increasing temporal thresholds: $$\Delta t = 0, 1, ..., 20$$
Each detector is shown in a different color, with error bars showing the SEM across recordings.
The left colomn show results when `GT1` (_RA_) is used as the ground truth, while the right column shows results when `GT2` (_MN_) is used.

In [4]:
W, H = 1300, 450

multi_thresholds_figure = ch_sdt.multi_channel_figure(
    csdt_metrics,
    metric=METRIC,
    yaxis_title=r"d'", show_other_gt=True,
    error_bars='std', show_err_bands=False,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_hspace=0, subplots_vspace=0,
)

multi_thresholds_figure.update_layout(
    width=W, height=H,
    title=dict(text=EVENT_NAME + "s", y=0.975, x=0.5, xanchor='center'),
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',

    # remove axis grids
    xaxis=dict(showgrid=False, zeroline=False, showline=False), yaxis=dict(showgrid=False, zeroline=False, showline=False),
    xaxis2=dict(showgrid=False, zeroline=False, showline=False), yaxis2=dict(showgrid=False, zeroline=False, showline=False),
    xaxis3=dict(showgrid=False, zeroline=False, showline=False), yaxis3=dict(showgrid=False, zeroline=False, showline=False),
    xaxis4=dict(showgrid=False, zeroline=False, showline=False), yaxis4=dict(showgrid=False, zeroline=False, showline=False),

    # move legend to bottom
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.05),
    # showlegend=False,   # hide legend
    margin=dict(l=10, r=0, b=10, t=0, pad=0),
)
multi_thresholds_figure.layout.annotations = []   # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 6, '', True
# save_fig(multi_thresholds_figure, FIG_ID, PANEL_ID, f"{TITLE.lower()}-discrimination_multi_threshold-{METRIC.lower()}", IS_SUPP)
multi_thresholds_figure.show()

## Statistical Analysis
### Onsets

In [5]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc = ch_sdt.post_hoc_table(
    onset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([onset_statistics, onset_pvalues, onset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  48.897297  7.817292e-09    True
                 RA  47.438849  1.529473e-08    True
d_prime          MN  53.091892  1.124345e-09    True
                 RA  55.899281  3.050701e-10    True
f1               MN  50.648649  3.484147e-09    True
                 RA  51.928058  1.927978e-09    True
false_alarm_rate MN  50.072333  4.546690e-09    True
                 RA  47.393502  1.561683e-08    True
precision        MN  52.248649  1.661988e-09    True
                 RA  54.086486  7.086749e-10    True
recall           MN  51.573770  2.271524e-09    True
                 RA  51.489130  2.362247e-09    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.     ***       *        *
         RA      --    n.s.    n.s.    n.s.     ***       *        *
ivvt     MN  0.9880      --    n.s.    n.s.     ***     ***       **
         RA  0.9960      --    n.s.    n.s.     ***     ***       **
idt      MN  0.9661  0.5941      --    n.s.       *    n.s.     n.s.
         RA  0.9603  0.6708      --    n.s.       *    n.s.     n.s.
idvt     MN  0.9645  0.5879  1.0000      --       *    n.s.     n.s.
         RA  0.9398  0.6097  1.0000      --       *    n.s.     n.s.
engbert  MN  0.0003  0.0000  0.0238  0.0246      --    n.s.     n.s.
         RA  0.0003  0.0000  0.0250  0.0341      --    n.s.     n.s.
nh       MN  0.0153  0.0005  0.2563  0.2611  0.9830      --     n.s.
         RA  0.0153  0.0009  0.2733  0.3251  0.9810      --     n.s.
remodnav MN  0.0407  0.0018  0.4258  0.4318  0.9298  1.0000       --
         RA  0.0206  0.0014  0.3197  0.3755  0.9699  1.0000       --

### Offsets

In [6]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc = ch_sdt.post_hoc_table(
    offset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([offset_statistics, offset_pvalues, offset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  46.489209  2.365566e-08    True
                 RA  43.424460  9.612335e-08    True
d_prime          MN  56.330935  2.495232e-10    True
                 RA  58.446043  9.305162e-11    True
f1               MN  56.805755  2.000040e-10    True
                 RA  57.669065  1.337266e-10    True
false_alarm_rate MN  54.379061  6.186268e-10    True
                 RA  50.079855  4.530929e-09    True
precision        MN  48.474820  9.496585e-09    True
                 RA  47.093525  1.792460e-08    True
recall           MN  52.969259  1.190134e-09    True
                 RA  55.200000  4.223816e-10    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.     ***      **     n.s.
         RA      --    n.s.    n.s.    n.s.     ***      **     n.s.
ivvt     MN  0.9960      --    n.s.    n.s.     ***     ***        *
         RA  0.9975      --    n.s.    n.s.     ***     ***        *
idt      MN  0.5973  0.1969      --    n.s.    n.s.    n.s.     n.s.
         RA  0.5785  0.2093      --    n.s.    n.s.    n.s.     n.s.
idvt     MN  0.5407  0.1628  1.0000      --    n.s.    n.s.     n.s.
         RA  0.4471  0.1330  1.0000      --    n.s.    n.s.     n.s.
engbert  MN  0.0001  0.0000  0.1361  0.1664      --    n.s.     n.s.
         RA  0.0002  0.0000  0.1851  0.2783      --    n.s.     n.s.
nh       MN  0.0019  0.0001  0.4349  0.4904  0.9981      --     n.s.
         RA  0.0024  0.0001  0.4842  0.6159  0.9989      --     n.s.
remodnav MN  0.1628  0.0234  0.9923  0.9960  0.5470  0.8831       --
         RA  0.1212  0.0186  0.9856  0.9965  0.6945  0.9374       --

## Figure: Distribution @ Threshold
### (not in the paper)
This figure shows the distribution of `METRIC` at the predefined temporal `THRESHOLD`, for each detector across recordings.
The right-side of each violin is w.r.t. `GT1` (_RA_) as the ground truth annotator, and the left-side is w.r.t. `GT2` (_MN_).

In [7]:
W, H = 600, 450

onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

######################################
## COMBINE ONSET AND OFFSET FIGURES ##
######################################

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=[f"{EVENT_NAME} {peyes.constants.ONSET_STR}", f"{EVENT_NAME} {peyes.constants.OFFSET_STR}"]
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles

# FIG_ID, IS_SUPP = 5, False
# save_fig(discriminability_figure, FIG_ID, 'left', 'fixation-discriminability', IS_SUPP)
discriminability_figure.show()

## Final Figure
### (Shown in the paper)
- Combine Multi-Threshold line-plots with Single-Threshold violin-plots
- use only one GT annotator

In [8]:
WIDTH, HEIGHT = 1400, 800
COLUMN_TITLES = [f"{EVENT_NAME} {peyes.constants.ONSET_STR.capitalize()}s", f"{EVENT_NAME} {peyes.constants.OFFSET_STR.capitalize()}s",]


def _convert_line_traces(line_fig: go.Figure, new_fig: go.Figure, gt: str) -> go.Figure:
    if gt == "RA":
        other = "MN"
        allowed_yaxes = ["y", "y3"]
    elif gt == "MN":
        other = "RA"
        allowed_yaxes = ["y2", "y4"]
    else:
        raise ValueError("Unknown GT: {}".format(gt))

    for trace in line_fig.data:
        if not trace['yaxis'] in allowed_yaxes:
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr["showlegend"] = False
        new_tr["line_width"] = LINE_WIDTH
        new_tr["error_y"]["thickness"] = ERROR_WIDTH
        new_tr["marker"]["size"] = LINE_WIDTH * 2

        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
            new_tr["marker"]["color"] = new_tr["line"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] in ['y', 'y2']   # onset traces are in top row
        new_fig.add_trace(new_tr, row=1, col=1 if is_onset else 2)
    return new_fig


def _convert_violin_traces(violin_fig, new_fig, gt: str) -> go.Figure:
    other = "MN" if gt == "RA" else "RA"
    for trace in violin_fig.data:
        if trace['scalegroup'] != METRIC:
            # ignore non `METRIC` traces
            continue
        if not trace["name"].startswith(gt):
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr['scalegroup'] = "VIOLINS"
        new_tr["showlegend"] = False

        # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
        if new_tr["x0"] == "Other GT":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
        elif new_tr["x0"].startswith("i"):
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
        elif new_tr["x0"] == "remodnav":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
        new_tr['x'] = new_tr['y']

        # reset irrelevant props
        new_tr['y'] = new_tr['x0'] = None
        new_tr["showlegend"] = False
        new_tr["side"] = "positive"
        new_tr["width"] = 1.8
        new_tr["box"] = dict(visible=False, width=0.95, line=dict(width=1))
        new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
        new_tr["opacity"] = 1
        new_tr["visible"] = True

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] == 'y'
        new_fig.add_trace(new_tr, row=2, col=1 if is_onset else 2)
    return new_fig


def convert_figs(line_fig, violin_fig, gt: str) -> go.Figure:
    # copy traces to new figure
    new_fig = make_subplots(
        rows=len(COLUMN_TITLES), cols=len(COLUMN_TITLES),
        shared_xaxes=False, shared_yaxes=True,
        vertical_spacing=0.12, horizontal_spacing=0.02,
        column_titles=COLUMN_TITLES,
    )
    _convert_line_traces(line_fig, new_fig, gt)
    _convert_violin_traces(violin_fig, new_fig, gt)

    # update legends
    new_fig.for_each_trace(lambda tr: tr.update(showlegend=tr['yaxis'] == 'y'))  # legend only shows one set of lines

    # update subtitles
    new_fig.for_each_annotation(lambda ann: ann.update(text=f"<b>{ann.text}</b>", font=TITLE_FONT, textangle=0, ))

    # add red rectangles in the top plots to to highlight the bottom subplots' origin
    for c in range(len(COLUMN_TITLES)):
        if gt=="RA":
            y0 = -0.75 if c == 0 else -0.55
            y1 = 4.85 if c == 0 else 5.25
        else:
            y0 = -0.65 if c == 0 else -0.65
            y1 = 4.75 if c == 0 else 5.3
        new_fig.add_shape(
            row=1, col=c + 1,
            type="rect",
            x0=THRESHOLD-0.5, x1=THRESHOLD+0.5,
            y0=y0, y1=y1,
            line_color="red", line_width=LINE_WIDTH,
        )

    # add axis labels
    for r in range(len(COLUMN_TITLES)):
        xaxis_title = "Δt (samples)" if r == 0 else r"$d'$"
        yaxis_title = r"$d'$" if r == 0 else "Detector"
        for c in range(len(COLUMN_TITLES)):
            new_fig.update_xaxes(
                row=r + 1, col=c + 1,
                title=dict(text=xaxis_title, font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=r == 0, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                tickfont=AXIS_TICK_FONT,
            )
            new_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=yaxis_title if c == 0 else "", font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                showticklabels=r==0 and c==0, tickfont=AXIS_TICK_FONT, tickangle=0 if r==0 else 30,
            )

    # add row annotations "A" and "B"
    for ann in ["A", "B"]:
        new_fig.add_annotation(
            text=f"<b>{ann}</b>", font={**TITLE_FONT, **dict(size=36)}, showarrow=False,
            xref="paper", yref="paper", xanchor="right", yanchor="top", x=0, y=1.0525 if ann == "A" else 0.46
        )


    new_fig.update_layout(
        font_family=FONT_FAMILY,
        width=WIDTH, height=HEIGHT,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            orientation="h", bgcolor='rgba(0, 0, 0, 0)',
            yanchor="top", y=-0.06, xanchor="center", x=0.5,
            font=AXIS_TICK_FONT, itemwidth=90,
        ),
    )
    return new_fig

In [9]:
GT = "RA"
NAME = f"fixation_boundary_sensitivity-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

In [10]:
GT = "MN"
NAME = f"fixation_boundary_sensitivity-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()